In [ ]:
import pandas as pd
import numpy as np


l0 = 1000 # Initital pool of policyholders
age = 35  # Age of policyholder used for single cell
premium = 1200 # Cost of policy per year
claims = 500000 # Fixed annuity in case of death


# Loading data from reference tables
vbt_table = pd.read_excel("Export.xlsx", skiprows=23, nrows=79)

df_vbt = pd.DataFrame(vbt_table)
df_vbt_singlecell = df_vbt[df_vbt[r"Row\Column"] == age]
df_vbt_singlecell = df_vbt_singlecell.drop(columns=[r"Row\Column"]).iloc[:, 0:20]


#  Creating lapse rate dataframe using study data
lapse_rate = [0.1, 0.08] # For year 1 and 2
for n in range(3, 21):
  lapse_rate.append(0.05) # For year 3 through 20

df_lapse = pd.DataFrame([lapse_rate])


# Combining Mortality and Lapse Rate DataFrames and renaming index column
df_lapse.columns = df_vbt_singlecell.columns

df_combined = pd.concat([
    df_vbt_singlecell.reset_index(drop=True),
    df_lapse.reset_index(drop=True)
    ], axis=0)

df_combined.index = ["Mortality Rate", "Lapse Rate"]


# Calculating Survival Rate and Survivors

# Total lives lost due to Mortality and Lapses
df_combined.loc["Net Retention"] = (1 - df_combined.loc["Mortality Rate"]) * (1 - df_combined.loc["Lapse Rate"])
df_combined.loc["End Lives"] = l0 * df_combined.loc["Net Retention"].cumprod()

# Population at start of year
start_lives = [l0] + df_combined.loc["End Lives"].iloc[:-1].tolist()
df_combined.loc["Start Lives"] = start_lives

# Death count due to Mortality and Survivors without lapses
df_combined.loc["Death Count"] = df_combined.loc["Start Lives"] * df_combined.loc["Mortality Rate"]
df_combined.loc["Mid-Year Survivors"] = df_combined.loc["Start Lives"] - df_combined.loc["Death Count"]

# Lapse count each year
df_combined.loc["Lapse Count"] = df_combined.loc["Mid-Year Survivors"] * df_combined.loc["Lapse Rate"]

# print(f"{df_combined}\n")

In [ ]:
# Cash Flow Projections

df_cashflow = df_combined.loc[["Start Lives", "Death Count"]].copy()
df_cashflow.loc["Premium In"] = df_cashflow.loc["Start Lives"] * premium
df_cashflow.loc["Claims Out"] = df_cashflow.loc["Death Count"] * claims
df_cashflow.loc["Cash Flow"] = df_cashflow.loc["Premium In"] - df_cashflow.loc["Claims Out"]

cum_cf = df_cashflow.loc["Cash Flow"].cumsum()

print(df_cashflow)
print(f"\nCumulative cashflow: {cum_cf.iloc[-1]:.2f}$")
